In [1]:
from langchain_groq import ChatGroq

In [2]:
llm = ChatGroq(
    temperature=0, 
    groq_api_key='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', 
    model_name="llama-3.1-70b-versatile"
)
response = llm.invoke("The first person to land on moon was ...")
print(response.content)

The first person to land on the moon was Neil Armstrong. He stepped onto the lunar surface on July 20, 1969, as part of the Apollo 11 mission.


In [7]:
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://jobs.nike.com/job/R-30433?from=job%20search%20funnel")
page_data = loader.load().pop().page_content
print(page_data)

Apply for Senior Information Security Engineer, ITC

Search JobsSkip navigationSearch JobsNIKE, INC. JOBSContract JobsJoin The Talent CommunityLife @ NikeOverviewBenefitsBrandsOverviewJordanConverseTeamsOverviewAdministrative SupportAdvanced InnovationAir Manufacturing InnovationAviationCommunicationsCustomer ServiceDesignDigitalFacilitiesFinance & AccountingGovernment & Public AffairsHuman ResourcesInsights & AnalyticsLegalManufacturing & EngineeringMarketingMerchandisingPlanningPrivacyProcurementProduct Creation, Development & ManagementRetail CorporateRetail StoresSalesSocial & Community ImpactSports MarketingStrategic PlanningSupply Chain, Distribution & LogisticsSustainabilityTechnologyLocationsOverviewNike WHQNike New York HQEHQ: Hilversum, The NetherlandsELC: Laakdal, BelgiumGreater China HQDiversity, Equity & InclusionOverviewMilitary InclusionDisability InclusionIndigenous InclusionInternshipsTechnologySenior Information Security Engineer, ITCKarnataka, IndiaBecome a Part of t

In [8]:
from langchain_core.prompts import PromptTemplate

prompt_extract = PromptTemplate.from_template(
        """
        ### SCRAPED TEXT FROM WEBSITE:
        {page_data}
        ### INSTRUCTION:
        The scraped text is from the career's page of a website.
        Your job is to extract the job postings and return them in JSON format containing the 
        following keys: `role`, `experience`, `skills` and `description`.
        Only return the valid JSON.
        ### VALID JSON (NO PREAMBLE):    
        """
)

chain_extract = prompt_extract | llm 
res = chain_extract.invoke(input={'page_data':page_data})
type(res.content)


str

In [9]:
from langchain_core.output_parsers import JsonOutputParser

json_parser = JsonOutputParser()
json_res = json_parser.parse(res.content)
json_res

{'role': 'Senior Information Security Engineer, ITC',
 'experience': '5+ years',
 'skills': ['Endpoint Protection',
  'Web proxy security',
  'Email security',
  'IDS/IPS',
  'WAF',
  'PKI',
  'Advanced Threat Protection',
  'Cloud',
  'Automation',
  'Scripting',
  'Web proxy certifications',
  'Endpoint certifications'],
 'description': 'We’re looking for a Sr. Cybersecurity Engineer to provide hands-on technical and tactical expertise to a 24x7 team of dedicated security engineers focused on maintaining operational stability while reducing risk. You will work on a team of experienced security professionals who are passionate about their work and Nike.'}

In [10]:
type(json_res)

dict

In [11]:
import pandas as pd

df = pd.read_csv("my_portfolio.csv")
df

,Techstack,Links
0,"React, Node.js, MongoDB",https://example.com/react-portfolio
1,"Angular,.NET, SQL Server",https://example.com/angular-portfolio
2,"Vue.js, Ruby on Rails, PostgreSQL",https://example.com/vue-portfolio
3,"Python, Django, MySQL",https://example.com/python-portfolio
4,"Java, Spring Boot, Oracle",https://example.com/java-portfolio
5,"Flutter, Firebase, GraphQL",https://example.com/flutter-portfolio
6,"WordPress, PHP, MySQL",https://example.com/wordpress-portfolio
7,"Magento, PHP, MySQL",https://example.com/magento-portfolio
8,"React Native, Node.js, MongoDB",https://example.com/react-native-portfolio
9,"iOS, Swift, Core Data",https://example.com/ios-portfolio


In [12]:
import uuid
import chromadb

client = chromadb.PersistentClient('vectorstore')
collection = client.get_or_create_collection(name="portfolio")

if not collection.count():
    for _, row in df.iterrows():
        collection.add(documents=row["Techstack"],
                       metadatas={"links": row["Links"]},
                       ids=[str(uuid.uuid4())])

In [13]:
job = json_res
job['skills']

['Endpoint Protection',
 'Web proxy security',
 'Email security',
 'IDS/IPS',
 'WAF',
 'PKI',
 'Advanced Threat Protection',
 'Cloud',
 'Automation',
 'Scripting',
 'Web proxy certifications',
 'Endpoint certifications']

In [14]:
links = collection.query(query_texts=job['skills'], n_results=2).get('metadatas', [])
links

[[{'links': 'https://example.com/xamarin-portfolio'},
  {'links': 'https://example.com/full-stack-js-portfolio'}],
 [{'links': 'https://example.com/python-portfolio'},
  {'links': 'https://example.com/magento-portfolio'}],
 [{'links': 'https://example.com/magento-portfolio'},
  {'links': 'https://example.com/wordpress-portfolio'}],
 [{'links': 'https://example.com/magento-portfolio'},
  {'links': 'https://example.com/wordpress-portfolio'}],
 [{'links': 'https://example.com/xamarin-portfolio'},
  {'links': 'https://example.com/ml-python-portfolio'}],
 [{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/magento-portfolio'}],
 [{'links': 'https://example.com/ios-ar-portfolio'},
  {'links': 'https://example.com/android-portfolio'}],
 [{'links': 'https://example.com/xamarin-portfolio'},
  {'links': 'https://example.com/devops-portfolio'}],
 [{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/magento-portfolio'}],
 [{'li

In [15]:
job

{'role': 'Senior Information Security Engineer, ITC',
 'experience': '5+ years',
 'skills': ['Endpoint Protection',
  'Web proxy security',
  'Email security',
  'IDS/IPS',
  'WAF',
  'PKI',
  'Advanced Threat Protection',
  'Cloud',
  'Automation',
  'Scripting',
  'Web proxy certifications',
  'Endpoint certifications'],
 'description': 'We’re looking for a Sr. Cybersecurity Engineer to provide hands-on technical and tactical expertise to a 24x7 team of dedicated security engineers focused on maintaining operational stability while reducing risk. You will work on a team of experienced security professionals who are passionate about their work and Nike.'}

In [16]:
job = json_res
job['skills']

['Endpoint Protection',
 'Web proxy security',
 'Email security',
 'IDS/IPS',
 'WAF',
 'PKI',
 'Advanced Threat Protection',
 'Cloud',
 'Automation',
 'Scripting',
 'Web proxy certifications',
 'Endpoint certifications']

In [21]:
prompt_email = PromptTemplate.from_template(
        """
        ### JOB DESCRIPTION:
        {job_description}
        
        ### INSTRUCTION:
        You are Abdul Fazal, a business development executive at AtliQ. AtliQ is an AI & Software Consulting company dedicated to facilitating
        the seamless integration of business processes through automated tools. 
        Over our experience, we have empowered numerous enterprises with tailored solutions, fostering scalability, 
        process optimization, cost reduction, and heightened overall efficiency. 
        Your job is to write a cold email to the client regarding the job mentioned above describing the capability of AtliQ 
        in fulfilling their needs.
        Also add the most relevant ones from the following links to showcase Atliq's portfolio: {link_list}
        Remember you are Abdul Fazal, BDE at AtliQ. 
        Do not provide a preamble.
        ### EMAIL (NO PREAMBLE):
        
        """
        )

chain_email = prompt_email | llm
res = chain_email.invoke({"job_description": str(job), "link_list": links})
print(res.content)

Subject: Expert Cybersecurity Solutions for Nike's ITC Team

Dear Hiring Manager,

I came across the job posting for a Senior Information Security Engineer, ITC at Nike, and I'm excited to introduce AtliQ, an AI & Software Consulting company that can help fulfill your team's needs. With our expertise in automation, scripting, and cloud security, we believe we can provide the necessary support to maintain operational stability while reducing risk.

Our team of experienced security professionals has a proven track record of delivering tailored solutions that foster scalability, process optimization, cost reduction, and heightened overall efficiency. We've empowered numerous enterprises with our automated tools, and we're confident that we can do the same for Nike.

Some of our relevant portfolio highlights include:

* Automation and Scripting: https://example.com/python-portfolio
* Cloud Security: https://example.com/ml-python-portfolio
* DevOps: https://example.com/devops-portfolio

The